# `datetime`

In [1]:
import datetime

Working with dates and times is one of the biggest challenges in programming. Between dealing with time zones, daylight saving time, and different written date formats, it can be tough to keep track of which days and times you’re referencing. Fortunately, the built-in Python `datetime` module can help you manage the complex nature of dates and times.

In this section, you’ll learn:

- Why programming with dates and times is such a challenge
- Which functions are available in the Python `datetime` module
- How to print or read a date and time in a specific format
- How to do arithmetic with dates and times

## Programming With Dates and Times

If you’ve ever worked on software that needed to keep track of times across several geographic areas, then you probably have a sense of why programming with time can be such a pain. The fundamental disconnect is that computer programs prefer events that are perfectly ordered and regular, but the way in which most humans use and refer to time is highly irregular.

> **Note:** If you want to learn more about why time can be so complicated to deal with, then there are many great resources available on the web. Here are a few good places to start:
> 
> - [Computerphile: The Problem With Time & Timezones](https://www.youtube.com/watch?v=-5wpm-gesOY)
> - [Working With Time Zones: Everything You Wish You Didn’t Need to Know](https://www.youtube.com/watch?v=rz3D8VG_2TY)
> - [The Complexity of Time Data Programming](https://www.mojotech.com/blog/the-complexity-of-time-data-programming/)

One great example of this irregularity is [daylight saving time](https://en.wikipedia.org/wiki/Daylight_saving_time). In the United States and Canada, clocks are set forward by one hour on the second Sunday in March and set back by one hour on the first Sunday in November. However, this has only been the case since 2007. Prior to 2007, clocks were set forward on the first Sunday in April and set back on the last Sunday in October.

Things get even more complicated when you consider [time zones](https://en.wikipedia.org/wiki/Time_zone). Ideally, time zone boundaries would follow lines of longitude exactly. However, for historical and political reasons, time zone lines are rarely straight. Often, areas that are separated by large distances find themselves in the same time zone, and adjacent areas are in different time zones. There are some time zones out there with [pretty funky shapes](https://upload.wikimedia.org/wikipedia/commons/8/88/World_Time_Zones_Map.png).

## How Computers Count Time

Nearly all computers count time from an instant called the Unix epoch. This occurred on **January 1, 1970, at 00:00:00 UTC**. **UTC stands for Coordinated Universal Time** and refers to the time at a longitude of 0°. UTC is often also called **Greenwich Mean Time**, or GMT. UTC is not adjusted for daylight saving time, so it consistently keeps twenty-four hours in every day.

By definition, Unix time elapses at the same rate as UTC, so a one-second step in UTC corresponds to a one-second step in Unix time. You can usually figure out the date and time in UTC of any given instant since January 1, 1970, by counting the number of seconds since the Unix epoch, with the exception of leap seconds. Leap seconds are occasionally added to UTC to account for the slowing of the Earth’s rotation but are not added to Unix time.

> **Note:** There’s an interesting bug associated with Unix time. Since many older operating systems are 32-bit, they store the Unix time in a 32-bit signed integer.
>
> This means that at 03:14:07 on January 19, 2038, the integer will overflow, resulting in what’s known as the Year 2038 problem, or Y2038. Similar to the Y2K problem, Y2038 will need to be corrected to avoid catastrophic consequences for critical systems.

Nearly all programming languages, including Python, incorporate the concept of Unix time. Python’s standard library includes a module called `time` that can print the number of seconds since the Unix epoch:

In [103]:
import time
time.time()

1621903066.768729

In this example, you import the time module and execute `time()` to print the Unix time, or number of seconds (excluding leap seconds) since the epoch.

In addition to Unix time, computers need a way to convey time information to users. As you saw in the last example, Unix time is nearly impossible for a human to parse. Instead, Unix time is typically converted to UTC, which can then be converted into a local time using **time zone offsets**.

**The Internet Assigned Numbers Authority (IANA)** maintains a [database](https://www.iana.org/time-zones) of all of the values of time zone offsets. IANA also releases regular updates that include any changes in time zone offsets. This database is often included with your operating system, although certain applications may include an updated copy.

The database contains a copy of all the designated time zones and how many hours and minutes they’re offset from UTC. So, during the winter, when daylight saving time is not in effect, the US Eastern time zone has an offset of -05:00, or negative five hours from UTC. Other regions have different offsets, which may not be integer hours. The UTC offset for Nepal, for example, is +05:45, or positive five hours and forty-five minutes from UTC.

## How Standard Dates Can Be Reported

Unix time is how computers count time, but it would be incredibly inefficient for humans to determine the time by calculating the number of seconds from an arbitrary date. Instead, we work in terms of years, months, days, and so forth. But even with these conventions in place, another layer of complexity stems from the fact that different languages and cultures have different ways of writing the date.

For instance, in the United States, dates are usually written starting with the month, then the day, then the year. This means that January 31, 2020, is written as **01-31-2020**. This closely matches the long-form written version of the date.

However, most of Europe and many other areas write the date starting with the day, then the month, then the year. This means that January 31, 2020, is written as **31-01-2020**. These differences can cause all sorts of confusion when communicating across cultures.

To help avoid communication mistakes, the International Organization for Standardization (ISO) developed [ISO 8601](https://en.wikipedia.org/wiki/ISO_8601). This standard specifies that all dates should be written in order of most-to-least-significant data. This means the format is year, month, day, hour, minute, and second:

```
YYYY-MM-DD HH:MM:SS
```

In this example, `YYYY` represents a four-digit year, and `MM` and `DD` are the two-digit month and day, starting with a zero if necessary. After that, `HH`, `MM`, and `SS` represent the two-digit hours, minutes, and seconds, starting with a zero if necessary.

The advantage of this format is that the date can be represented with no ambiguity. Dates written as `DD-MM-YYYY` or `MM-DD-YYYY` can be misinterpreted if the day is a valid month number. You’ll see a little later on how you can use the ISO 8601 format with Python `datetime`.

## How Time Should Be Stored in Your Program

Most developers who have worked with time have heard the advice to convert local time to UTC and store that value for later reference. In many cases, especially when you’re storing dates from the past, this is enough information to do any necessary arithmetic.

However, a problem can happen if a user of your program inputs a future date in their local time. Time zone and daylight saving time rules change fairly frequently, as you saw earlier with the 2007 change in daylight saving time for the United States and Canada. If the time zone rules for your user’s location change before the future date that they inputted, then UTC won’t provide enough information to convert back to the correct local time.

> **Note:** There are a number of excellent resources available to help you determine the appropriate way to store time data in your application. Here are a few places to start:
> 
> - Daylight saving time and time zone best practices
> - Storing UTC is not a Silver Bullet
> - How to save datetimes for future events
> - Coding Best Practices Using DateTime in the .NET Framework

In this case, you need to store the local time, including the time zone, that the user inputted as well as the version of the IANA time zone database that was in effect when the user saved the time. This way, you’ll always be able to convert the local time to UTC. However, this approach won’t always allow you to convert UTC to the correct local time.

## Using the Python `datetime` Module

As you can see, working with dates and times in programming can be complicated. Fortunately, you rarely need to implement complicated features from scratch these days since many open-source libraries are available to help out. This is definitely the case in Python, which includes three separate modules in the standard library to work with dates and times:

1. [`calendar`](https://docs.python.org/3/library/calendar.html#module-calendar) outputs calendars and provides functions using an idealized Gregorian calendar.
2. [`datetime`](https://docs.python.org/3/library/datetime.html) supplies classes for manipulating dates and times.
3. [`time`](https://docs.python.org/3/library/time.html) provides time-related functions where dates are not needed.

In this tutorial, you’ll focus on using the Python `datetime` module. The main focus of datetime is to make it less complicated to access attributes of the object related to dates, times, and time zones. Since these objects are so useful, calendar also returns instances of classes from datetime.

`time` is less powerful and more complicated to use than `datetime`. Many functions in time return a special `struct_time` instance. This object has a named tuple interface for accessing stored data, making it similar to an instance of `datetime`. However, it doesn’t support all of the features of datetime, especially the ability to perform arithmetic with time values.

`datetime` provides three classes that make up the high-level interface that most people will use:

- [`datetime.date`](https://docs.python.org/3/library/datetime.html#date-objects): is an idealized date that assumes the Gregorian calendar extends infinitely into the future and past. This object stores the year, month, and day as attributes.
- [`datetime.time`](https://docs.python.org/3/library/datetime.html#time-objects): is an idealized time that assumes there are 86,400 seconds per day with no leap seconds. This object stores the hour, minute, second, microsecond, and tzinfo (time zone information).
- [`datetime.datetime`](https://docs.python.org/3/library/datetime.html#datetime-objects): is a combination of a date and a time. It has all the attributes of both classes.

## Creating Python datetime Instances

The three classes that represent dates and times in `datetime` have similar initializers. They can be instantiated by passing keyword arguments for each of the attributes, such as year, date, or hour. You can try the code below to get a sense of how each object is created:

In [1]:
from datetime import date, time, datetime

In [2]:
date(year=2020, month=1, day=31)

datetime.date(2020, 1, 31)

In [3]:
time(hour=13, minute=14, second=31)

datetime.time(13, 14, 31)

In [4]:
datetime(year=2020, month=1, day=31, hour=13, minute=14, second=31)

datetime.datetime(2020, 1, 31, 13, 14, 31)

In this code, you import the three main classes from datetime and instantiate each of them by passing arguments to the constructor. You can see that this code is somewhat verbose, and if you don’t have the information you need as integers, these techniques can’t be used to create datetime instances.

Fortunately, datetime provides several other convenient ways to create `datetime` instances. These methods don’t require you to use integers to specify each attribute, but instead allow you to use some other information:

- `date.today()`: creates a datetime.date instance with the current local date.
- `datetime.now()`: creates a datetime.datetime instance with the current local date and time.
- `datetime.combine()`: combines instances of datetime.date and datetime.time into a single datetime.datetime instance.

These three ways of creating datetime instances are helpful when you don’t know in advance what information you need to pass into the basic initializers. You can try out this code to see how the alternate initializers work:

In [7]:
today = date.today()

today

datetime.date(2021, 5, 30)

In [8]:
now = datetime.now()
now

datetime.datetime(2021, 5, 30, 19, 40, 29, 641456)

In [9]:
current_time = time(now.hour, now.minute, now.second)
datetime.combine(today, current_time)

datetime.datetime(2021, 5, 30, 19, 40, 29)

In this code, you use `date.today()`, `datetime.now()`, and `datetime.combine()` to create instances of date, datetime, and time objects. Each instance is stored in a different variable:

1. **`today`** is a `date` instance that has only the year, month, and day.
2. **`now`** is a `datetime` instance that has the year, month, day, hour, minute, second, and microseconds.
3. **`current_time`** is a `time` instance that has the hour, minute, and second set to the same values as now.

On the last line, you combine the `date` information in `today` with the `time` information in `current_time` to produce a new `datetime` instance.

> **Warning:** `datetime` also provides `datetime.utcnow()`, which returns an instance of datetime at the current UTC. However, the Python documentation recommends against using this method because it doesn’t include any time zone information in the resulting instance.
>
> Using `datetime.utcnow()` may produce some surprising results when doing arithmetic or comparisons between datetime instances. In a later section, you’ll see how to assign time zone information to `datetime` instances.

## Using Strings to Create Python `datetime` Instances

Another way to create date instances is to use [`.fromisoformat()`](https://docs.python.org/3/library/datetime.html#datetime.date.fromisoformat). To use this method, you provide a string with the date in the ISO 8601 format that you learned about earlier. For instance, you might provide a string with the year, month, and date specified:

```
2020-01-31
```

This string represents the date January 31, 2020, according to the ISO 8601 format. You can create a `date` instance with the following example

In [10]:
date.fromisoformat("2020-01-31")

datetime.date(2020, 1, 31)

In this code, you use `date.fromisoformat()` to create a date instance for January 31, 2020. This method is very useful because it’s based on the ISO 8601 standard. But what if you have a string that represents a date and time but isn’t in the ISO 8601 format?

Fortunately, Python datetime provides a method called `.strptime()` to handle this situation. This method uses a special mini-language to tell Python which parts of the string are associated with the `datetime` attributes.

To construct a `datetime` from a string using `.strptime()`, you have to tell Python what each of the parts of the string represents using formatting codes from the mini-language. You can try this example to see how `.strptime()` works:

In [15]:
date_string = "01-31-2020 14:45:37"
format_string = "%m-%d-%Y %H:%M:%S"

On **line 1**, you create `date_string`, which represents the date and time **January 31, 2020, at 2:45:37 PM**. On **line 2**, you create `format_string`, which uses the mini-language to specify how the parts of `date_string` will be turned into `datetime` attributes.

In format_string, you include several formatting codes and all of the **dashes (-)**, **colons (:)**, and **spaces** exactly as they appear in `date_string`. To process the date and time in date_string, you include the following formatting codes:


|Component|Code|Value|
|:--|:--|:--|
|Year (as four-digit integer)|%Y|2020|
|Month (as zero-padded decimal)|%m|01|
|Date (as zero-padded decimal)|%d|31|
|Hour (as zero-padded decimal with 24-hour clock)|%H|14|
|Minute (as zero-padded decimal)|%M|45|
|Second (as zero-padded decimal)|%S|37|

A complete listing of all of the options in the mini-language is outside the scope of this tutorial, but you can find several good references on the web, including in Python’s [documentation](https://docs.python.org/3/library/datetime.html#strftime-strptime-behavior) and on a website called [strftime.org](https://strftime.org/).

Now that date_string and format_string are defined, you can use them to create a datetime instance. Here’s an example of how `.strptime()` works:

In [16]:
datetime.strptime(date_string, format_string)

datetime.datetime(2020, 1, 31, 14, 45, 37)

> **Note:** There are more advanced ways to create datetime instances, but they involve using third-party libraries that must be installed. One particularly neat library is called `dateparser`, which allows you to provide natural language string inputs. The input is even supported in a number of languages:
>
> ```python
>>> import dateparser
>>> dateparser.parse("yesterday")
datetime.datetime(2020, 3, 13, 14, 39, 1, 350918)
>>> dateparser.parse("morgen")
datetime.datetime(2020, 3, 15, 14, 39, 7, 314754)
>```
>
> In this code, you use `dateparser` to create two datetime instances by passing two different string representations of time. On line 1, you import dateparser. Then, on line 2, you use `.parse()` with the argument "yesterday" to create a datetime instance twenty-four hours in the past. At the time of writing, this was March 13, 2020, at 2:39 PM.
>
> On line 3, you use `.parse()` with the argument "morgen". Morgen is the German word for tomorrow, so dateparser creates a `datetime` instance twenty-four hours in the future. At the time of writing, this was March 15 at 2:39 PM.

## Starting Your PyCon Countdown

Now you have enough information to start working on a countdown clock for next year’s [PyCon US](https://us.pycon.org/)! PyCon US 2021 will start on May 12, 2021 in Pittsburgh, PA. With the 2020 event having been canceled, many Pythonistas are extra excited for next year’s gathering. This is a great way to keep track of how long you’ll need to wait and boost your `datetime` skills at the same time!

To get started, create a file called `pyconcd.py` and add this code:

```python
# pyconcd.py

from datetime import datetime

PYCON_DATE = datetime(year=2022, month=5, day=12, hour=8)
countdown = PYCON_DATE - datetime.now()
print(f"Countdown to PyCon US 2021: {countdown}")
```

In [76]:
from datetime import datetime

PYCON_DATE = datetime(year=2022, month=5, day=12, hour=8)

In [77]:
countdown = PYCON_DATE - datetime.now()
print(f"Countdown to PyCon US 2021: {countdown}")

Countdown to PyCon US 2021: 346 days, 12:01:47.847346


In this code, you import datetime from datetime and define a constant, `PYCON_DATE`, that stores the date of the next PyCon US. You don’t expect the date of PyCon to change, so you name the variable in all caps to indicate that it’s a constant.

Next, you compute the difference between `datetime.now()`, which is the current time, and `PYCON_DATE`. Taking the difference between two datetime instances returns a [datetime.timedelta](https://docs.python.org/3/library/datetime.html#timedelta-objects) instance.

timedelta instances represent the change in time between two datetime instances. The delta in the name is a reference to the Greek letter delta, which is used in science and engineering to mean a change. You’ll learn more later about how to use timedelta for more general arithmetic operations.

Finally the printed output, as of April 9, 2020 at a little before 9:30 PM is:

```
Countdown to PyCon US 2021: 397 days, 10:35:32.139350
```

Only 397 days until PyCon US 2021! This output is a little clunky, so later on you’ll see how you can improve the formatting. If you run this script on a different day, you’ll get a different output. If you run the script after May 12, 2021 at 8:00 AM, you’ll get a negative amount of time remaining!

## Working With Time Zones

As you saw earlier, storing the time zone in which a date occurs is an important aspect of ensuring your code is correct. Python `datetime` provides `tzinfo`, which is an abstract base class that allows `datetime.datetime` and `datetime.time` to include time zone information, including an idea of daylight saving time.

However, `datetime` does not provide a direct way to interact with the IANA time zone database. The Python `datetime.tzinfo` documentation recommends using a third-party package called `dateutil`. You can install `dateutil` with `pip`:

```shell
$ pip install python-dateutil
```

> Note that the name of the package that you install from PyPI, python-dateutil, is different from the name that you use to import the package, which is just dateutil.

## Using `dateutil` to Add Time Zones to Python `datetime`

One reason that `dateutil` is so useful is that it includes an interface to the IANA time zone database. This takes the hassle out of assigning time zones to your `datetime` instances. Try out this example to see how to set a `datetime` instance to have your local time zone:

In [23]:
from dateutil import tz
from datetime import datetime

In [36]:
now = datetime.now(tz=tz.tzlocal())

In [37]:
now

datetime.datetime(2021, 5, 30, 19, 49, 56, 431662, tzinfo=tzlocal())

In [38]:
now.tzname()

'EDT'

In this example, you import `tz` from `dateutil` and `datetime` from `datetime`. You then create a `datetime` instance set to the current time using `.now()`.

You also pass the `tz` keyword to `.now()` and set `tz` equal to `tz.tzlocal()`. In `dateutil`, `tz.tzlocal()` returns a concrete instance of `datetime.tzinfo`. This means that it can represent all the necessary time zone offset and daylight saving time information that datetime needs.

You can also create time zones that are not the same as the time zone reported by your computer. To do this, you’ll use `tz.gettz()` and pass the official [IANA name](https://en.wikipedia.org/wiki/List_of_tz_database_time_zones) for the time zone you’re interested in. Here’s an example of how to use `tz.gettz()`:

In [177]:
from dateutil import tz
from datetime import datetime

In [43]:
edm_tz = tz.gettz("America/Edmonton")

In [44]:
edm_tz

tzfile('/usr/share/zoneinfo/UTC')

In [41]:
now = datetime.now(tz=edm_tz)

In [42]:
now.tzname()

'MDT'

In [199]:
now

datetime.datetime(2021, 5, 24, 19, 4, 1, 437460, tzinfo=tzfile('/usr/share/zoneinfo/America/Edmonton'))

In [200]:
London_tz = tz.gettz("Europe/London")
now = datetime.now(tz=London_tz)

In [201]:
now

datetime.datetime(2021, 5, 25, 2, 4, 7, 661204, tzinfo=tzfile('/usr/share/zoneinfo/Europe/London'))

In [202]:
now.tzname()

'BST'

In this example, you use `tz.gettz()` to retrieve the time zone information for **London, United Kingdom** and store it in `London_tz`. You then retrieve the current time, setting the time zone to `London_tz`.

On Windows, this gives the `tzinfo` attribute the value `tzfile('GB-Eire')`. On macOS or Linux, the tzinfo attribute will look something like `tzfile('/usr/share/zoneinfo/Europe/London)`, but it might be slightly different depending on where `dateutil` pulls the time zone data from.

In an earlier section, you learned that you shouldn’t use `.utcnow()` to create a `datetime` instance at the current UTC. Now you know how to use `dateutil.tz` to supply a time zone to the datetime instance. Here’s an example modified from the recommendation in the Python documentation:

In [47]:
datetime.now(tz=tz.UTC)

datetime.datetime(2021, 5, 30, 23, 51, 25, 520443, tzinfo=tzutc())

In this code, you use `tz.UTC` to set the time zone of `datetime.now()` to the UTC time zone. This method is recommended over using `utcnow()` because `utcnow()` returns a **naive** `datetime` instance, whereas the method demonstrated here returns an **aware** `datetime` instance.

In [209]:
datetime.now(tz=tz.UTC)

datetime.datetime(2021, 5, 25, 1, 5, 55, 460639, tzinfo=tzutc())

Next, you’ll take a small detour to learn about **naive vs aware** `datetime` instances. If you already know all about this, then you can skip ahead to improve your PyCon countdown with time zone information

### Comparing Naive and Aware Python datetime Instances

Python `datetime` instances support two types of operation, naive and aware. The basic difference between them is that naive instances don’t contain time zone information, whereas aware instances do. More formally, to quote the Python documentation:

> An aware object represents a specific moment in time that is not open to interpretation. A naive object does not contain enough information to unambiguously locate itself relative to other date/time objects. ([Source](https://docs.python.org/3/library/datetime.html#id1))

This is an important distinction for working with Python `datetime`. An aware `datetime` instance can compare itself unambiguously to other aware `datetime` instances and will always return the correct time interval when used in arithmetic operations.

Naive `datetime` instances, on the other hand, may be ambiguous. One example of this ambiguity relates to daylight saving time. Areas that practice daylight saving time turn the clocks forward one hour in the spring and backward one hour in the fall. This typically happens at 2:00 AM local time. In the spring, the hour from 2:00 AM to 2:59 AM never happens, and in the fall, the hour from 1:00 AM to 1:59 AM happens twice!

Practically, what happens is that the offset from UTC in these time zones changes throughout the year. IANA tracks these changes and catalogs them in the different database files that your computer has installed. Using a library like `dateutil`, which uses the IANA database under the hood, is a great way to make sure that your code properly handles arithmetic with time.

> **Note:** In Python, the difference between naive and aware datetime instances is determined by the `tzinfo` attribute. An aware `datetime` instance has the `tzinfo` attribute equal to a subclass of the `datetime.tzinfo` abstract base class.
> 
> Python 3.8 and below provide one concrete implementation of `tzinfo` called `timezone`. However, timezone is limited to expressing fixed offsets from UTC that cannot change throughout the year, so it isn’t that useful when you need to account for changes such as daylight saving time.
> 
> Python 3.9 includes a new module called [`zoneinfo`](https://docs.python.org/3.9/library/zoneinfo.html) that provides a concrete implementation of `tzinfo` that tracks the IANA database, so it includes changes like daylight saving time. However, until Python 3.9 becomes widely used, it probably makes sense to rely on `dateutil` if you need to support multiple Python versions.
> 
> `dateutil` also provides several concrete implementations of tzinfo in the tz module that you used earlier. You can check out the [`dateutil.tz` documentation](https://dateutil.readthedocs.io/en/stable/tz.html) for more information.

This doesn’t mean that you always need to use aware datetime instances. But aware instances are crucial if you’re comparing times with each other, especially if you’re comparing times in different parts of the world.

## Improving Your PyCon Countdown


Now that you know how to add time zone information to a Python datetime instance, you can improve your PyCon countdown code. Earlier, you used the standard datetime constructor to pass the year, month, day, and hour that PyCon will start. You can update your code to use the [`dateutil.parser`](https://dateutil.readthedocs.io/en/stable/parser.html) module, which provides a more natural interface for creating `datetime` instances:

In [79]:
# pyconcd.py

from dateutil import parser, tz
from datetime import datetime

In [80]:
PYCON_DATE = parser.parse("May 12, 2022 8:00 AM")

In [81]:
PYCON_DATE = PYCON_DATE.replace(tzinfo=tz.gettz("America/New_York"))

In [82]:
now = datetime.now(tz=tz.tzlocal())

In [84]:
countdown = PYCON_DATE - now
print(f"Countdown to PyCon US 2021: {countdown}")

Countdown to PyCon US 2021: 346 days, 12:01:01.911105


In this code, you import `parser` and `tz` from `dateutil` and `datetime` from `datetime`. Next, you use `parser.parse()` to read the date of the next PyCon US from a string. This is much more readable than the plain datetime constructor.

`parser.parse()` returns a naive `datetime` instance, so you use `.replace()` to change the `tzinfo` to the **America/New_York** time zone. PyCon US 2021 will take place in Pittsburgh, Pennsylvania, which is in the US Eastern time zone. The canonical name for that time zone is **America/New_York**.

`PYCON_DATE` is an aware `datetime` instance with the time zone set to US Eastern time. Since May 12 is after daylight saving time takes effect, the time zone name is 'EDT', or 'Eastern Daylight Time'.

Next, you create now to represent the current instant of time and give it your local time zone. Last, you find the `timedelta` between `PYCON_DATE` and now and print the result. If you’re in a locale that does not adjust the clocks for daylight saving time, then you may see the number of hours remaining until PyCon change by an hour.

## Doing Arithmetic With Python datetime

Python `datetime` instances support several types of arithmetic. As you saw earlier, this relies on using `timedelta `instances to represent time intervals. `timedelta` is very useful because it’s built into the Python standard library. Here’s an example of how to work with `timedelta`:

In [86]:
from datetime import datetime, timedelta

In [88]:
now = datetime.now()
now

datetime.datetime(2021, 5, 30, 20, 0, 26, 901895)

In [91]:
tomorrow = timedelta(days=+1)

In [92]:
now + tomorrow

datetime.datetime(2021, 5, 31, 20, 0, 26, 901895)

In this code, you create `now`, which stores the current time, and `tomorrow`, which is a `timedelta` of `+1` days. Next, you add `now` and `tomorrow` to produce a `datetime` instance one day in the future. Note that working with naive `datetime` instances, as you are here, means that the day attribute of the `datetime` increments by one and does not account for any repeated or skipped time intervals.

`timedelta` instances also support negative values as the input to the arguments:

In [93]:
yesterday = timedelta(days=-1)
now + yesterday

datetime.datetime(2021, 5, 29, 20, 0, 26, 901895)

In this example, you provide `-1` as the input to `timedelta`, so when you add `now` and `yesterday`, the result is a decrease by one in the days attribute.

`timedelta` instances support addition and subtraction as well as positive and negative integers for all arguments. You can even provide a mix of positive and negative arguments. For instance, you might want to add three days and subtract four hours:

In [95]:
delta = timedelta(days=+3, hours=-4)
now + delta

datetime.datetime(2021, 6, 2, 16, 0, 26, 901895)

In this example, you add three days and subtract four hours, so the new `datetime` is at **January 29 at 5:37 AM**. `timedelta` is very useful in this way, but it’s somewhat limited because it cannot add or subtract intervals larger than a day, such as a month or a year. Fortunately, `dateutil` provides a more powerful replacement called `relativedelta`.

The basic syntax of `relativedelta` is very similar to `timedelta`. You can provide keyword arguments that produce changes of any number of years, months, days, hours, seconds, or microseconds. You can reproduce the first `timedelta` example with this code:

In [97]:
from dateutil.relativedelta import relativedelta

In [98]:
tomorrow = relativedelta(days=+1)
now + tomorrow

datetime.datetime(2021, 5, 31, 20, 0, 26, 901895)

In this example, you use `relativedelta` instead of `timedelta` to find the `datetime` corresponding to tomorrow. Now you can try adding five years, one month, and three days to now while subtracting four hours and thirty minutes:

In [100]:
delta = relativedelta(years=+5, months=+1, days=+3, hours=-4, minutes=-30)
now + delta

datetime.datetime(2026, 7, 3, 15, 30, 26, 901895)

You can also use `relativedelta` to calculate the difference between two `datetime` instances. Earlier, you used the subtraction operator to find the difference between two Python `datetime` instances, `PYCON_DATE` and `now`. With `relativedelta`, instead of using the subtraction operator, you need to pass the two `datetime` instances as arguments

In [82]:
now

datetime.datetime(2021, 5, 24, 18, 9, 33, 564827)

In [83]:
tomorrow = datetime(2020, 1, 27, 9, 37, 46, 380905)

In [84]:
relativedelta(now, tomorrow)

relativedelta(years=+1, months=+3, days=+27, hours=+8, minutes=+31, seconds=+47, microseconds=+183922)

In this example, you create a new `datetime` instance for `tomorrow` by incrementing the `days` field by one. Then, you use `relativedelta` and pass `now` and `tomorrow` as the two arguments. `dateutil` then takes the difference between these two `datetime` instances and returns the result as a `relativedelta` instance. 

`dateutil.relativedelta` objects have countless other uses. You can use them to find complex calendar information, such as the next year in which October the 13th falls on a Friday or what the date will be on the last Friday of the current month. You can even use them to replace attributes of a datetime instance and create, for example, a `datetime` one week in the future at 10:00 AM. You can read all about these other uses in the `dateutil` [documentation](https://dateutil.readthedocs.io/en/stable/examples.html#relativedelta-examples).

## Finishing Your PyCon Countdown

You now have enough tools in your belt to finish your PyCon 2021 countdown clock and provide a nice interface to use as well. In this section, you’ll use `relativedelta` to calculate the time remaining until PyCon, develop a function to print the time remaining in a nice format, and show the date of PyCon to the user.

### Using `relativedelta` in Your PyCon Countdown

First, replace the plain subtraction operator with `relativedelta`. With the subtraction operator, your `timedelta` object couldn’t count intervals of time larger than a day. However, `relativedelta` allows you to show the years, months, and days remaining:

In [101]:
# pyconcd.py

from dateutil import parser, tz
from dateutil.relativedelta import relativedelta
from datetime import datetime

In [103]:
PYCON_DATE = parser.parse("May 12, 2022 8:00 AM")
PYCON_DATE = PYCON_DATE.replace(tzinfo=tz.gettz("America/New_York"))
now = datetime.now(tz=tz.tzlocal())

In [104]:
now - PYCON_DATE 

datetime.timedelta(days=-347, seconds=43445, microseconds=152910)

In [106]:
countdown = relativedelta(PYCON_DATE, now)
print(f"Countdown to PyCon US 2021: {countdown}")

Countdown to PyCon US 2021: relativedelta(months=+11, days=+11, hours=+11, minutes=+55, seconds=+54, microseconds=+847090)


The only change that you made in this code was to replace line 11 with `countdown = relativedelta(PYCON_DATE, now)`. 

However, that output isn’t very pretty since it looks like the signature of `relativedelta()`. You can build up some prettier output by customizing it.

## Showing the PyCon Date in Your PyCon Countdown

Earlier, you learned about creating datetime instances using `.strptime()`. This method uses a special mini-language within Python to specify how the date string is formatted.

Python `datetime` has an additional method called `.strftime()` that allows you to format a `datetime` instance to a string. In a sense, it’s the reverse operation of parsing using `.strptime()`. You can differentiate between the two methods by remembering that the `p` in `.strptime()` stands for **parse**, and the `f` in `.strftime()` stands for **format**.

In your PyCon countdown, you can use `.strftime()` to print output to let the user know the date on which PyCon US will start. Remember, you can find the formatting codes that you want to use on [strftime.org](https://strftime.org/). Now add this code on line 18 of your PyCon countdown script:

In [130]:
pycon_date_str = PYCON_DATE.strftime("%A, %B %d, %Y at %H:%M %p %Z")
print(f"PyCon US 2021 will start on:", pycon_date_str)

PyCon US 2021 will start on: Thursday, May 12, 2022 at 08:00 AM EDT


## Alternatives to Python `datetime` and `dateutil`

Python `datetime` and `dateutil` are a powerful combination of libraries when you’re working with dates and times. `dateutil` is even recommended in the Python documentation.

However, there are many other libraries that you can use to work with dates and times in Python. Some of these rely on `datetime` and `dateutil`, while others are completely independent replacements:

- [`pytz`](https://pypi.org/project/pytz/): provides time zone information similar to dateutil. It uses a somewhat different interface than the standard datetime.tzinfo, so be aware of the potential problems if you decide to use it.
- [`Arrow`](https://arrow.readthedocs.io/en/latest/): provides a drop-in replacement for datetime. It’s inspired by moment.js, so if you’re coming from web development, then this might be a more familiar interface.
- [`Pendulum`](https://pendulum.eustace.io/): provides another drop-in replacement for datetime. It includes a time zone interface and an improved timedelta implementation.
- [`Maya`](https://github.com/timofurrer/maya): provides a similar interface as datetime. It relies on Pendulum for parts of the parsing library.
- [`dateparser`](https://dateparser.readthedocs.io/en/latest/): provides an interface to generate datetime instances from human-readable text. It’s flexible and supports many languages.

In addition, if you work heavily with [NumPy](https://realpython.com/tutorials/numpy/), [Pandas](https://realpython.com/courses/introduction-pandas-and-vincent/), or other data science packages, then there are a few options that might be useful to you:

- [`NumPy`](https://numpy.org/doc/1.18/reference/arrays.datetime.html): provides a similar API to the built-in Python datetime library, but the NumPy version can be used in arrays.
- [`Pandas`](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html): provides support for time-series data in DataFrames, usually sequential values of time-based events, by using the NumPy datetime module.
- [`cftime`](https://unidata.github.io/cftime/api.html): provides support for calendars other than the [proleptic Gregorian calendar](https://en.wikipedia.org/wiki/Proleptic_Gregorian_calendar) as well as other time units conforming to the Climate and Forecasting (CF) conventions. It’s used by the [`xarray`](http://xarray.pydata.org/en/stable/time-series.html) package to provide time-series support.

## Further Reading

Since programming with time can be so complicated, there are many resources on the web to help you learn more about it. Fortunately, this is a problem that many people who work in every programming language have thought about, so you can usually find information or tools to help with any problem you may have. Here’s a selected list of articles and videos that I found helpful in writing this tutorial:


- [Daylight saving time and time zone best practices](https://stackoverflow.com/a/2532962)
- [Storing UTC is not a Silver Bullet](https://codeblog.jonskeet.uk/2019/03/27/storing-utc-is-not-a-silver-bullet/)
- [How to save datetimes for future events](http://www.creativedeletion.com/2015/03/19/persisting_future_datetimes.html)
- [Coding Best Practices Using DateTime in the .NET Framework](https://docs.microsoft.com/en-us/previous-versions/dotnet/articles/ms973825(v=msdn.10))
- [Computerphile: The Problem with Time & Timezones](https://www.youtube.com/watch?v=-5wpm-gesOY)
- [The Complexity of Time Data Programming](https://www.mojotech.com/blog/the-complexity-of-time-data-programming/)

In addition, Paul Ganssle is a core contributor to CPython and the current maintainer of dateutil. His articles and videos are a great resource for Python users:

- [Working with Time Zones: Everything You Wish You Didn’t Need to Know (PyCon 2019)](https://www.youtube.com/watch?v=rz3D8VG_2TY)
- [pytz: The Fastest Footgun in the West](https://blog.ganssle.io/articles/2018/03/pytz-fastest-footgun.html)
- [Stop using utcnow and utcfromtimestamp](https://blog.ganssle.io/articles/2019/11/utcnow.html)
- [A curious case of non-transitive datetime comparison](https://blog.ganssle.io/articles/2018/02/a-curious-case-datetimes.html)

## Conclusion

In this tutorial, you learned about programming with dates and times and why it often leads to errors and confusion. You also learned about the Python `datetime` and `dateutil` modules as well as how to work with time zones in your code.

Now you can:

- **Store** dates in a good, future-proof format in your programs
- **Create** Python `datetime` instances with formatted strings
- **Add** time zone information to `datetime` instances with `dateutil`
- **Perform arithmetic operations** with `datetime` instances using `relativedelta`